# Chat API

* Connecting to Claude's API via [claudette](https://claudette.answer.ai/) for chat completion, 
* Extracting JSON and rules to populate gradio UI

In [1]:
#| default_exp chat

In [14]:
#| hide
from nbdev.showdoc import *

In [15]:
#| export
import claudette
from pathlib import Path
import re

* Claudette provides models, which is a list of models currently available from the SDK.

In [16]:
[print(m) for m in claudette.models]
# using Sonnet 3.5
model = claudette.models[3]
print(f'\nChosen model: {model}')

claude-3-opus-20240229
claude-3-7-sonnet-20250219
claude-3-5-sonnet-20241022
claude-3-haiku-20240307
claude-3-5-haiku-20241022

Chosen model: claude-3-haiku-20240307


In [17]:
import os
api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key: print('key exists ☑')

key exists ☑


In [45]:
context_file = Path('docs/analytics.md')
context_media = context_file.read_text(encoding='utf-8')

system_prompt = """You are a helpful and concise assistant."""

chat = claudette.Chat(model, sp=system_prompt)
response = chat("""Can you come up with 10 realistic questions a hiring manager might ask in an opening interview?  the job title for this role is: SENIOR RESEARCH ANALYST,
               Do not add any additional commentary, just list 10 questions only. Start each question with a new line starting like:
                [Q1] ... 
                [Q2] ...
                My resume is in the project docs, the core competencies for a senior analyst are below:
                {context_media}""")

response

[Q1] Can you walk me through your relevant work experience as a research analyst?

[Q2] How do you stay up-to-date with the latest industry trends and developments?

[Q3] Can you describe a challenging research project you've worked on and how you approached it?

[Q4] What data analysis tools and techniques are you most proficient in?

[Q5] How do you ensure the accuracy and reliability of your research findings?

[Q6] Can you share an example of a time when you had to present complex research to a non-technical audience?

[Q7] How do you prioritize and manage multiple research projects simultaneously?

[Q8] What do you consider to be the most important qualities for a successful senior research analyst?

[Q9] How do you collaborate with cross-functional teams to gather and synthesize information?

[Q10] Can you describe your experience in using data visualization techniques to communicate research insights?

<details>

- id: `msg_01NrwmTSrBCVfQXF9uMy12Zg`
- content: `[{'citations': None, 'text': "[Q1] Can you walk me through your relevant work experience as a research analyst?\n\n[Q2] How do you stay up-to-date with the latest industry trends and developments?\n\n[Q3] Can you describe a challenging research project you've worked on and how you approached it?\n\n[Q4] What data analysis tools and techniques are you most proficient in?\n\n[Q5] How do you ensure the accuracy and reliability of your research findings?\n\n[Q6] Can you share an example of a time when you had to present complex research to a non-technical audience?\n\n[Q7] How do you prioritize and manage multiple research projects simultaneously?\n\n[Q8] What do you consider to be the most important qualities for a successful senior research analyst?\n\n[Q9] How do you collaborate with cross-functional teams to gather and synthesize information?\n\n[Q10] Can you describe your experience in using data visualization techniques to communicate research insights?", 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 122, 'output_tokens': 208}`

</details>

In [46]:
response_text = response.content[0].text  
questions = re.findall(r'^\s*\d+\.\s+(.+)$', response_text, re.MULTILINE)
questions

[]

In [42]:
q1 = questions[0]
q1

'Can you walk me through your experience as a senior research analyst?'

## More directly testing on Mock Interviews:

In [47]:
INTERVIEW_QS = response


In [48]:
INTERVIEW_INSTRUCTIONS = """Can you walk through each of the interview question from the. file, one by one, and I 
will reply directly to them in the response, as in a mock interview. I want you to set up the following sections 
with line breaks between them 

----- ANSWER EVAL ----- {Summarise user input answer concisely and into a few bullet points, (depending on length). 
This section will be displayed in Markdown, so format appropriately.
Use headings GOOD, with a small numbered list of what is considered high-quality response. 
Use heading BAD, critique of where the answer went wrong, think about verbosity, clarity, what kind of thing should be 
answered in a high level interview, for a very senior position like this.]

[RESPONSE METRICS] Here please return a json object with these metrics as keys and a score rating between 0-10 (1 d.p.). 
This is mostly for fun but do the best you can with it. The metrics:
* Linguistic Clarity
* Confidence
* Technical Expertise
* Strategic Thinking
* Impact Demonstration
* Industry Understanding
[ENDJSON]

----- REWRITEN ----- 
Use the above notes to write a first-in-class script for the question. Use the same general facts as the actual answer, 
but don't be afraid to add in external context to the answer if your descretion requires it. This is a fictional exercise 
and we want the answer to be an example of a perfect answer. Use bold to highly key words and phrases in the answer, and 
keep it conversational in tone. It needs to be spoken out loud, so not like and essay. 
----- END ----- 
If the users answers the question in the response the evaluate it like above, and then ask to move on to the next question. 
For each question and answer repeat the same instructions."""

In [50]:
chat(INTERVIEW_INSTRUCTIONS)

[Q1] Can you walk me through your relevant work experience as a research analyst?

<details>

- id: `msg_01VH1TznMT2qRNb1pSPX7ZZD`
- content: `[{'citations': None, 'text': '[Q1] Can you walk me through your relevant work experience as a research analyst?', 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 766, 'output_tokens': 21}`

</details>

In [52]:
chat("Yeah, when I was working as a research analyst, it was pretty tight. Lots of people asked me some pretty hard questions and I, you know, I just crushed it")

----- ANSWER EVAL -----

**GOOD**
1. The response conveys a sense of confidence in handling challenging questions and tasks as a research analyst.

**BAD**
1. The language used is very informal and lacks the level of professionalism expected for a senior-level position.
2. The response does not provide any specific details about the candidate's work experience, accomplishments, or the types of projects they have worked on.
3. There is no demonstration of the candidate's technical expertise, strategic thinking, or industry understanding.

[RESPONSE METRICS]
{
  "Linguistic Clarity": 2.0,
  "Confidence": 4.0,
  "Technical Expertise": 2.0,
  "Strategic Thinking": 2.0,
  "Impact Demonstration": 1.0,
  "Industry Understanding": 2.0
}

----- REWRITTEN -----

**As a senior research analyst, I've had the opportunity to work on a variety of complex projects that have allowed me to develop a deep understanding of the industry and hone my analytical skills.** One of the key projects I worked on was a comprehensive analysis of the competitive landscape in our target market. This involved gathering data from multiple sources, including industry reports, customer surveys, and competitor websites, and then synthesizing the information to identify key trends, challenges, and opportunities.

**I was able to present my findings to the executive team in a clear and concise manner, which ultimately led to the development of a successful market entry strategy.** Throughout this process, I had to handle a number of challenging questions from stakeholders, and I was able to draw upon my strong technical expertise and strategic thinking to provide well-informed and impactful responses.

**In addition to my technical skills, I've also developed strong communication and collaboration abilities, which have been essential in working with cross-functional teams to ensure that our research insights are effectively integrated into the decision-making process.** I believe these skills, combined with my deep industry knowledge and analytical capabilities, make me well-suited for a senior research analyst role.

<details>

- id: `msg_01WgYLhkodCiAsLx7xBZe5Fh`
- content: `[{'citations': None, 'text': '----- ANSWER EVAL -----\n\n**GOOD**\n1. The response conveys a sense of confidence in handling challenging questions and tasks as a research analyst.\n\n**BAD**\n1. The language used is very informal and lacks the level of professionalism expected for a senior-level position.\n2. The response does not provide any specific details about the candidate\'s work experience, accomplishments, or the types of projects they have worked on.\n3. There is no demonstration of the candidate\'s technical expertise, strategic thinking, or industry understanding.\n\n[RESPONSE METRICS]\n{\n  "Linguistic Clarity": 2.0,\n  "Confidence": 4.0,\n  "Technical Expertise": 2.0,\n  "Strategic Thinking": 2.0,\n  "Impact Demonstration": 1.0,\n  "Industry Understanding": 2.0\n}\n\n----- REWRITTEN -----\n\n**As a senior research analyst, I\'ve had the opportunity to work on a variety of complex projects that have allowed me to develop a deep understanding of the industry and hone my analytical skills.** One of the key projects I worked on was a comprehensive analysis of the competitive landscape in our target market. This involved gathering data from multiple sources, including industry reports, customer surveys, and competitor websites, and then synthesizing the information to identify key trends, challenges, and opportunities.\n\n**I was able to present my findings to the executive team in a clear and concise manner, which ultimately led to the development of a successful market entry strategy.** Throughout this process, I had to handle a number of challenging questions from stakeholders, and I was able to draw upon my strong technical expertise and strategic thinking to provide well-informed and impactful responses.\n\n**In addition to my technical skills, I\'ve also developed strong communication and collaboration abilities, which have been essential in working with cross-functional teams to ensure that our research insights are effectively integrated into the decision-making process.** I believe these skills, combined with my deep industry knowledge and analytical capabilities, make me well-suited for a senior research analyst role.', 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 1345, 'output_tokens': 457}`

</details>

# Gradio UI

In [60]:

import gradio as gr

def respond(message, history):
    return chat(message)


# Create a Gradio interface with the Chatbot component
chatbot = gr.ChatInterface(
    fn=respond,
    examples=["Hello Neo.. let's begin."],
    theme="default"
)

# Launch the interface inline in the Jupyter notebook
chatbot.launch(inline=True)  # The inline=True parameter is key for Jupyter integration

d:\ProgramData\Anaconda3.9\envs\wingman\Lib\site-packages\gradio\components\chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


: 

In [51]:
#| hide
import nbdev; nbdev.nbdev_export()